# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [2]:
from openai import OpenAI

client = OpenAI(api_key='sk-u1KVKPjooxeYYhsm318b42F4922047B8846c3f0bA33bA59c',base_url='https://api.xiaoai.plus/v1')

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一个宁静的自然风景。画面中心是一条木质栈道，它穿过一片茂盛的绿色草地，直通远方。栈道的两旁是高高的草丛，草地的远处则有一些低矮的树木和灌木。图像背景是一片开阔的天空，蓝天中布满了细碎的白云，给画面增添了几分宁静和广阔的感觉。\n\n这种场景可能位于一个自然保护区或是公园内，是很适合散步和自然观察的地点。整体上，这张图片传达了一种平静和回归自然的氛围。', role='assistant', function_call=None, tool_calls=None))


In [3]:
response.choices[0].message.content

'这幅图展示了一个宁静的自然风景。画面中心是一条木质栈道，它穿过一片茂盛的绿色草地，直通远方。栈道的两旁是高高的草丛，草地的远处则有一些低矮的树木和灌木。图像背景是一片开阔的天空，蓝天中布满了细碎的白云，给画面增添了几分宁静和广阔的感觉。\n\n这种场景可能位于一个自然保护区或是公园内，是很适合散步和自然观察的地点。整体上，这张图片传达了一种平静和回归自然的氛围。'

### 封装成一个函数 query_image_description

In [7]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(api_key='sk-u1KVKPjooxeYYhsm318b42F4922047B8846c3f0bA33bA59c',base_url='https://api.xiaoai.plus/v1')
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [18]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图使用幽默的方式对比了两种不同的生活方式或状态，并以柴犬为主角。左边的柴犬被赋予了一副夸张的肌肉人的身体，带有这样的文字说明：“16岁的我，工作后的我”。这暗示在16岁时充满了活力和力量。旁边附有的文字可能在描述一天的精力充沛、学业上的积极和外貌上的自信。

右边的柴犬代表了工作后的状态，显示出一种更为放松甚至懒散的体态，脸上的表情也显得较为无奈。与之相对应的文字提到了常感觉累、对待工作的态度以及外貌上的变化。

这种对比通常用来幽默地表达许多人从学生时代到成年后工作生活中的巨大转变，尤其是体能和心态上的变化。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [20]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(api_key='sk-u1KVKPjooxeYYhsm318b42F4922047B8846c3f0bA33bA59c',base_url='https://api.xiaoai.plus/v1')

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [22]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图表展示了1980年到2020年间，美国、中国、日本和德国的国内生产总值（GDP）的比较。横轴表示时间，从1980年到2020年；纵轴表示GDP的值，单位是万亿美元。

- 蓝色折线代表美国的GDP，始终保持在最高位，显示出持续增长的趋势。
- 红色折线代表中国的GDP，从1990年代开始显著上升，2000年以后增长速度非常快，到2020年接近美国的水平。
- 紫色折线代表日本的GDP，增长相对平稳，1995年后略有下滑，之后基本保持稳定。
- 绿色折线代表德国的GDP，整体增长较缓慢且稳定。

总体上，图表显示了四个国家的经济增长情况，特别突出了中国的快速增长，以及这些国家GDP之间的对比和变化趋势。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [23]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图显示的是一本笔记本上记录的一些深度学习和机器学习的技术和方法的笔记。内容主要涵盖了几种用于自然语言处理（NLP）的模型调整技术，具体包括：

1. **Prompt Tuning**：介绍了Prompt Tuning技术，用于调整小型模型。这种技术涉及到将输入的文本X（由一系列子符号X1, X2,..., Xn组成）进行嵌入转换，生成适合模型处理的格式。

2. **Prefix Tuning**：这部分讨论了Prefix Tuning策略，其中引入了额外的参数W'，和原有的W一起形成更新后的模型矩阵，进行前缀式的参数调整以适应特定的任务。

3. **LoRA（Low-Rank Adaptation）**：介绍了LoRA技术，是一种只对模型权重进行局部低秩调整的方法。主要通过修改权重矩阵W，加入一个低秩调整ΔW，从而减少需要调整的参数数量，提高模型调整的效率和降低资源消耗。

笔记中还涉及了模型效果的一些比较数据，比如提到使用LoRA进行调整时模型的参数量的变化等。

整体来看，这张图片反映的是关于如何使用现代深度学习技术对自然处理语言模型进行优化和调整的一些高级技术和思想。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [24]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图显示的是一本笔记本上记录的一些深度学习和机器学习的技术和方法的笔记。内容主要涵盖了几种用于自然语言处理（NLP）的模型调整技术，具体包括：

1. **Prompt Tuning**：介绍了Prompt Tuning技术，用于调整小型模型。这种技术涉及到将输入的文本X（由一系列子符号X1, X2,..., Xn组成）进行嵌入转换，生成适合模型处理的格式。

2. **Prefix Tuning**：这部分讨论了Prefix Tuning策略，其中引入了额外的参数W'，和原有的W一起形成更新后的模型矩阵，进行前缀式的参数调整以适应特定的任务。

3. **LoRA（Low-Rank Adaptation）**：介绍了LoRA技术，是一种只对模型权重进行局部低秩调整的方法。主要通过修改权重矩阵W，加入一个低秩调整ΔW，从而减少需要调整的参数数量，提高模型调整的效率和降低资源消耗。

笔记中还涉及了模型效果的一些比较数据，比如提到使用LoRA进行调整时模型的参数量的变化等。

整体来看，这张图片反映的是关于如何使用现代深度学习技术对自然处理语言模型进行优化和调整的一些高级技术和思想。

![](./images/handwriting_1.jpg)

In [25]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一页笔记，主要内容关注的是人工智能领域中的深度学习技术，特别是关于变换模型（Transformers）的训练和微调技术。笔记上用中文和英文混杂记录，内容涉及以下几个部分：

1. **Transformers**：提到了与变换模型相关的基本概念及其在性能基准方面的应用。

2. **PEFT & SOTA**：讨论了前沿的技术性能（State of the Art）和相关的PEFT方法。

3. **Prompt Tuning**：这是一种用于微调语言模型的技术，笔记中提到了几种不同的Prompt Tuning方法，包括Adapter, Prefix, Prompt Tuning等技术，以及它们的来源（如Google, Stanford）。

4. **Instruction Tuning & FT-LLMs**：涉及到针对大型语言模型（LLMs）的指令式微调。

5. **MAM Adapters** 和 **Refinement in LLMs**：这部分提到了微调的专门技术，如MAM Adapter和在大型语言模型中的精细化技术。

6. **模型列表**：如ChatGPT, BLOOM, 和GPT3等，这显示笔记作者可能在探索或比较这些模型的特性和性能。

整体来看，这些笔记显示出作者在深入研究如何通过各种微调技术提高深度学习模型的性能。这些内容对于从事人工智能研究或开发的人来说非常有参考价值。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [27]:
from IPython.display import display, Markdown

content = query_base64_image_description("./images/ai_training.jpg")
# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))


这张图片中使用红色笔迹写有中文以及英文内容。中文内容是“我说OPEN AI”，而英文则是日期“2024/6/11”。整体来看，这可能是一份关于OPEN AI的笔记或声明，并标注了具体的日期。这些内容可能是记录者在特定日期对于OPEN AI的看法或计划。